# Database Query Checker
Scratchpad for testing queries on the database

In [1]:
from pprint import pprint
import pandas as pd
from RecommenderAgent import WhiskyRecommender
import sqlite3
self = WhiskyRecommender()
alchemy_path = self.alchemy
sqlite_path = self.dbpath
kw_path = self.kwpath
con_to_db = self.dbcon

Database files available. Agent initialising.
Agent loading complete.


In [3]:
self.updateWhiskies()

Getting whisky's from https://www.masterofmalt.com/new-arrivals/whisky-new-arrivals/1.
Getting the following attributes:
['ID',
 'Type',
 'Name',
 'Description',
 'Nose',
 'Palate',
 'Finish',
 'Price',
 'Size',
 'Abv',
 'URL']
- Getting details of Invergordon 33 Year Old 1987 (cask 88794) - The Cooper's Choice (The Vintage Malt Whisky Co.).
- Getting details of From The Sample Room Sweet & Smoky - The Cooper's Choice (The Vintage Malt Whisky Co.).
- Getting details of Dumbarton 20 Year Old 2000 (cask 211097) -  The Cooper's Choice (The Vintage Malt Whisky Co.).
- Getting details of Finglassie Lowland Smoke (cask 409) - The Cooper's Choice (The Vintage Malt Whisky Co.).
- Getting details of Deanston 11 Year Old 2009 (cask 5211) - The Cooper's Choice (The Vintage Malt Whisky Co,).
- Getting details of Caol Ila 8 Year Old 2012 (cask 331912) -  The Cooper's Choice (The Vintage Malt Whisky Co.).
- Getting details of Kilnaughton Secret Islay (cask 279) - The Cooper's Choice (The Vintage Mal

In [182]:
tns = self.getTastingNotes()
reviews = self.allReviewsConcatDF()
self.performKWExtraction(tns)


Performing Nose Keyword Extraction
Building Corpus
Building Graph
Candidate Keywords Selected


KeyboardInterrupt: 

In [4]:
kws = self.loadKWs()

In [5]:
mom_model = self.vectorizeNotesMatrix(tns, kws)

Vectorising Nose tasting notes
Vectorising Palate tasting notes
Vectorising Finish tasting notes
Vectorising All tasting notes


In [86]:
reviews_model = self.vectorizeNotesMatrix(reviews, kws)

Vectorising Nose tasting notes
Vectorising Palate tasting notes
Vectorising Finish tasting notes
Vectorising All tasting notes


In [169]:
THRESH = 50


mom = mom_model["general"]
rev = reviews_model["general"]

# List of features
cols = mom.drop(columns="ID").columns

# Setting ID as index
mom.index=mom.ID
rev.index = rev.ID

# Selecting number of reviews from reviews dataframe, and finding weighting factor
n = reviews[["n"]]
n["n"] = n["n"].apply(lambda n: min(1, n/THRESH))


<ipython-input-169-8648d44af090>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n["n"] = n["n"].apply(lambda n: min(1, n/THRESH))


In [171]:
# Adding weighted reviews to mom vectors, and re-normalising
rev[cols] = rev[cols].add(mom[cols]).dropna().apply(lambda row: normVec(row), axis=1)

In [175]:
# Replacing relevant notes with notes including reviews
mom.loc[rev.index] = rev

In [179]:
mom = mom.drop(columns="ID").reset_index()

In [180]:
mom

,ID,still,jam,join,citrus,apricot,ice,develop,peat,initially,...,melt,menthol,tropical,wine,syrup,lead,rise,mixed,tart,bring
0,a6f4f91bc7350f8dfa43f628422fc248,0.000000,0.000000,0.000000,0.0,0.127000,0.127000,0.127,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1683d020e998066f77efcd30a00b37fd,0.000000,0.169031,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,de36426070013546d87c307ce8e94b75,0.000000,0.000000,0.137361,0.0,0.000000,0.137361,0.000,0.137361,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,72e09be39ee18cbf7142198885a18b5c,0.185695,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.557086,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.185695,0.0,0.0
4,33b773397906abf7a778a5b82da6832b,0.000000,0.000000,0.141421,0.0,0.141421,0.000000,0.000,0.141421,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15136,c51c317945a468bcd6ff0aa764d01d1b,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
15137,f7aaee22c9fd5b15ab420922e3b26ef4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
15138,47e83c880cc3563141f670e6c0a0e094,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
15139,d839d4d73cea6dfb07e588f4acc050e4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [96]:
rev[cols] = rev[cols].multiply(n["n"], axis="index")

In [113]:
out = mom[cols].add(rev[cols], fill_value=0)

In [115]:
# Useful vector functions:
def normVec(v):
    norm = np.linalg.norm(v)
    if norm != 0:
        v = v / norm
    return v

In [118]:
out = out.reset_index()

In [126]:
out[cols].loc[0,:]

still      0.000000
jam        0.208514
join       0.000000
citrus     0.000000
apricot    0.000000
             ...   
lead       0.000000
rise       0.000000
mixed      0.000000
tart       0.000000
bring      0.000000
Name: 0, Length: 260, dtype: float64

In [129]:
out[cols].apply(lambda row: normVec(row), axis=1)

,still,jam,join,citrus,apricot,ice,develop,peat,initially,long,...,melt,menthol,tropical,wine,syrup,lead,rise,mixed,tart,bring
0,0.0,0.208514,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.417029,0.0,0.0,0.0,0.0,0.0
1,0.0,0.145865,0.0,0.0,0.0,0.000000,0.000000,0.437595,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.4,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.213201,...,0.0,0.0,0.0,0.213201,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15136,0.0,0.000000,0.0,0.0,0.0,0.000000,0.308607,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
15137,0.0,0.000000,0.0,0.0,0.0,0.000000,0.185695,0.557086,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
15138,0.0,0.000000,0.0,0.0,0.0,0.179605,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
15139,0.0,0.000000,0.0,0.0,0.0,0.213201,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [140]:
rev[cols].add(mom[cols]).dropna()

,still,jam,join,citrus,apricot,ice,develop,peat,initially,long,...,melt,menthol,tropical,wine,syrup,lead,rise,mixed,tart,bring
ID,,,,,,,,,,,,,,,,,,,,,
52a3bcfe44bddf29bf258bff6352ba12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.127000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
add5f25217c80ce15eed058443aec29c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.404906,0.0,0.132453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
with_reviews = rev[cols].add(mom[cols]).dropna().apply(lambda row: normVec(row), axis=1)
with_reviews

,still,jam,join,citrus,apricot,ice,develop,peat,initially,long,...,melt,menthol,tropical,wine,syrup,lead,rise,mixed,tart,bring
ID,,,,,,,,,,,,,,,,,,,,,
52a3bcfe44bddf29bf258bff6352ba12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.127000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
add5f25217c80ce15eed058443aec29c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.38716,0.0,0.126648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
mom.loc[with_reviews.index]

,still,jam,join,citrus,apricot,ice,develop,peat,initially,long,...,menthol,tropical,wine,syrup,lead,rise,mixed,tart,bring,ID
ID,,,,,,,,,,,,,,,,,,,,,
52a3bcfe44bddf29bf258bff6352ba12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.127000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52a3bcfe44bddf29bf258bff6352ba12
add5f25217c80ce15eed058443aec29c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.264906,0.0,0.132453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add5f25217c80ce15eed058443aec29c
